In [ ]:
#require "core";;
#require "yaml";;
#require "fpath"
#require "yaml.unix"
#require "lwt"
#require "cohttp"
#require "cohttp-lwt-unix"
#require "yojson"
#require "ppx_yojson_conv"
#use "/home/opam/work/lib/util.ml"


In [ ]:
open Lwt
open Cohttp
open Cohttp_lwt_unix


let keyvallist_to_form_urlencoded p =
  p |> List.map (fun (k,v) -> Printf.sprintf "%s=%s" k v)
  |> String.concat "&"
  |> Cohttp_lwt.Body.of_string

let keyvallist_to_form_json p = p
  |> List.map (fun (k,v) -> Printf.sprintf {|"%s": "%s"|} k v)
  |> String.concat ", "
  |> Printf.sprintf "{%s}"
  |> Cohttp_lwt.Body.of_string

let (alpaca_secret, alpaca_key) = 
  let yaml = Yaml_unix.of_file_exn (Fpath.v "keys.paper.yaml") in
  let key = yaml |> get_value "key" |> get_string_opt |> get_string in 
  let secret = yaml |> get_value "secret" |> get_string_opt |> get_string in
  (secret, key)

let headers = Header.init ()
  |> fun h -> Header.add_list h 
              [("APCA-API-KEY-ID", alpaca_key); 
               ("APCA-API-SECRET-KEY", alpaca_secret)]
            
let order_list = [("symbol", "MSFT"); ("qyt","10"); ("side","buy"); ("type","market"); ("time_in_force","day")]

let test_post =
  let uri = Uri.of_string "https://paper-api.alpaca.markets/v2/orders" in
  let body = keyvallist_to_form_json order_list in
    Cohttp_lwt_unix.Client.post ~headers ~body uri >>= fun (resp, body) ->
      let code = resp |> Response.status |> Code.code_of_status in
      Printf.printf "Response code: %d\n" code;
      Printf.printf "Headers: %s\n" (resp |> Response.headers |> Header.to_string);
      body |> Cohttp_lwt.Body.to_string >|= fun body ->
      Printf.printf "Body of length: %d\n" (String.length body);
      body

let () =
  let respBody = Lwt_main.run test_post in
  print_endline (respBody)

In [ ]:
let body = keyvallist_to_form_json order_list 

In [ ]:
  let body = keyvallist_to_form_urlencoded order_list in

In [20]:
type order = {
  stock: string;
  qty: int;
} [@@deriving yojson]

type order = { stock : string; qty : int; }
val order_to_yojson : order -> Yojson.Safe.t = <fun>
val order_of_yojson :
  Yojson.Safe.t -> order Ppx_deriving_yojson_runtime.error_or = <fun>
val _ : Yojson.Safe.t -> order Ppx_deriving_yojson_runtime.error_or = <fun>


In [ ]:
let msft_order = {
  stock= "MSFT";
  qty= 10
}

In [ ]:
let o1 = order_to_yojson msft_order 

In [21]:
let o2 = o1 |> Yojson.Safe.to_string

val o2 : string = "{\"stock\":\"MSFT\",\"qty\":10}"
